In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp
from sklearn import preprocessing
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f).transpose()
    return labels

In [9]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [10]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [11]:
training_data_x = pickle.load(open("TS-Train.pkl", "rb")).dropna(axis=1)
training_data_x = preprocessing.normalize(training_data_x,norm='max', axis=0)
training_data_x = training_data_x.transpose()
print(training_data_x.shape)
training_data_y = labels("../Labels-Train.npy")
print(training_data_y)

#set hyperparameters & variables
learning_rate = 0.03
epochs = 500
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

(769, 12589)
[[1 0 0 ... 0 0 1]]


In [12]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [13]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })
    
    loss_arr.append(current_loss)
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W,training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-11-21 03:12:32.712045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 03:12:32.712663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 03:12:32.713103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 03:12:32.713576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 03:12:32.713983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Optimization Finished!
Training error= 0.044522736 Weights= [[ 5.01339557e-03  1.71137024e-02  2.23325472e-02 -7.04786414e-03
  -1.28422189e-03  4.18380456e-04 -2.31175637e-03 -3.91539419e-03
   2.71226210e-03  0.00000000e+00 -1.28422189e-03 -8.17341171e-03
  -1.37613516e-03 -1.86205478e-04  3.30962153e-04 -1.78718846e-02
  -6.13529934e-03 -1.17396191e-03 -2.31175637e-03 -2.60637645e-02
  -7.80727118e-02  1.90359354e-02 -2.83349510e-02  6.65879948e-03
   6.86497753e-03 -2.16727750e-03 -1.82716665e-03 -5.60510624e-03
  -1.99350878e-03 -7.80419679e-03 -9.05426778e-03 -2.37163925e-03
  -1.78080739e-03  8.17341171e-03 -1.13108251e-02  1.78207687e-04
  -1.60381795e-04 -4.39446121e-05  2.58842076e-04  2.90711585e-04
   3.44454194e-04  9.12157632e-03 -1.02082058e-03  0.00000000e+00
  -4.55699973e-02  5.99909425e-02 -4.52779094e-03 -8.16772226e-03
  -8.17341171e-03 -8.17341171e-03 -8.17341171e-03 -8.17341171e-03
  -8.17341171e-03 -8.17341171e-03 -8.17341171e-03 -8.17341171e-03
  -8.17341171e-0

In [14]:
test_x = pickle.load(open("TS-Test.pkl", "rb")).dropna(axis=1)
test_x = preprocessing.normalize(test_x, norm='max', axis=0)
test_x = test_x.transpose()

test_y = labels("../Labels-Test.npy")

predicted_y = np.dot(session.run(W), test_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  46.8869123252859 %


In [ ]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
